In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from tqdm import tqdm
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 


from stable_baselines import PPO2
from codes.environment.multichannel import SimpleCavityEnv
from stable_baselines.common.vec_env import SubprocVecEnv
import numpy as np
import qutip as qt
import matplotlib
import matplotlib.pyplot as plt
from codes.functions.check_policy import load_info
import os
import json
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
import os
from mpl_toolkits.axes_grid1.inset_locator import mark_inset, inset_axes, InsetPosition, BboxPatch, BboxConnector
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
plt.rcParams.update({'font.size': 10})
plt.rcParams.update({'figure.dpi': 150})
import seaborn as sns
def make_env(args, i):
    def _init():
        env = SimpleCavityEnv(args, counter=i,testing=True)
        return env
    return _init

import matplotlib
matplotlib.use('module://ipykernel.pylab.backend_inline')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
folder="training/2021-03-02_fock_new_v8"

num_subfolders=len([x for x in sorted(os.listdir(folder))[2:] if ""in x  ])
models=np.zeros((num_subfolders), dtype=object)
args=np.zeros((num_subfolders), dtype=object)
summaries=np.zeros((num_subfolders), dtype=object)

for index in range(num_subfolders):
    direc=folder+"/training_"+str(index)
    args[index] = load_info(direc, print=False)
    args[index]["t_mean"]=10
    last_model="rl_model_"+ str(sorted([int(x.split("_")[2]) for x in os.listdir(direc+"/model")])[30])+"_steps"
    models[index]=PPO2.load(direc+"/model/"+last_model)
    if os.path.isfile(folder+"/summaries/summary_training_0.npy"):
        summaries[index]=np.load(folder+"/summaries/summary_training_"+str(index)+ ".npy",allow_pickle=True).item()
        
np.save("data/summaries_focks.npy", summaries)        
    

Loading a model without an environment, this model cannot be trained until it has a valid environment.
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Loading a model without an environment, this model cannot be trained until it has a valid environment.


FileNotFoundError: [Errno 2] No such file or directory: 'data/summaries_focks.npy'

In [ ]:
ntraj=100
direc = folder+"/training_0"
args[0] = load_info(direc, print=False)


args[0]

In [4]:
ntraj=100
direc = folder+"/training_0"
args[0] = load_info(direc, print=False)
args[0]["t_mean"]=10
args[0]["T_max"]=args[index]["T_max"]*5
args[0]["timesteps"]=args[index]["timesteps"]*5
args[0]["multiplier"]=args[index]["multiplier"]*5

env = SubprocVecEnv([make_env(args[0], i) for i in range(ntraj)], start_method="fork")
timesteps=env.get_attr("T")[0]
Nstates=env.get_attr("Nstates")[0]
substeps=env.get_attr("numberPhysicsMicroSteps")[0]
num_actions=env.get_attr("num_actions")[0]


rhos=np.zeros((num_subfolders,ntraj, timesteps, Nstates, Nstates),dtype="complex")
probabilities=np.zeros((num_subfolders,ntraj,Nstates,timesteps*substeps))
actions=np.zeros((num_subfolders,ntraj,num_actions,timesteps))
fidelities=np.zeros((num_subfolders,ntraj,timesteps))
env.close()


for index, model in enumerate(models):
    direc = folder+"/training_"+str(index)
    args[index] = load_info(direc, print=False)
    args[index]["t_mean"]=10
    args[index]["T_max"]=args[index]["T_max"]*5
    args[index]["timesteps"]=args[index]["timesteps"]*5
    args[index]["multiplier"]=args[index]["multiplier"]*5
    
    
    
    env = SubprocVecEnv([make_env(args[index], i) for i in range(ntraj)], start_method="fork")
    
    
    
    
    timesteps=env.get_attr("T")[0]
    Nstates=env.get_attr("Nstates")[0]
    substeps=env.get_attr("numberPhysicsMicroSteps")[0]
    num_actions=env.get_attr("num_actions")[0]
    
    

    
    
    print(index)
    obs=env.reset()
    for i in tqdm(range(args[index]["timesteps"])):
        action=models[index].predict(obs, deterministic=True)
        rhos[index, :, i]=env.get_attr("Rho")
        obs, r, done, _ =env.step(action[0])


    probabilities[index]=np.array(env.get_attr("probabilities"))
    actions[index]=np.array(env.get_attr("actions_plot"))
    fidelities[index]=np.array(env.get_attr("fidelities"))
    env.close()
    #print(np.mean(fidelities[index, :, -1], axis=0))

    
np.save("data/rhos_focks.npy", rhos)
np.save("data/probabilities_focks.npy", probabilities)
np.save("data/actions_focks.npy", actions)
np.save("data/fidelities_focks.npy", fidelities)


0


100%|██████████| 2500/2500 [04:34<00:00,  9.10it/s]


1


100%|██████████| 2500/2500 [04:41<00:00,  8.89it/s]


2


...

100%|██████████| 2500/2500 [04:51<00:00,  8.58it/s]


5


100%|██████████| 2500/2500 [04:32<00:00,  9.17it/s]


6


100%|██████████| 2500/2500 [04:51<00:00,  8.57it/s]


In [5]:
folder="training/2021-03-02_fock_3_multiple"

num_subfolders=len([x for x in sorted(os.listdir(folder))[2:] if ""in x  ])
models_fock3=np.zeros((num_subfolders), dtype=object)
args_fock3=np.zeros((num_subfolders), dtype=object)
summaries_fock3=np.zeros((num_subfolders), dtype=object)

for index in range(num_subfolders):
    direc=folder+"/training_"+str(index)
    args_fock3[index] = load_info(direc, print=False)
    args_fock3[index]["t_mean"]=10
    last_model="rl_model_"+ str(sorted([int(x.split("_")[2]) for x in os.listdir(direc+"/model")])[-1])+"_steps"
    models_fock3[index]=PPO2.load(direc+"/model/"+last_model)
    summaries_fock3[index]=np.load(folder+"/summaries/summary_training_"+str(index)+ ".npy",allow_pickle=True).item()
 
np.save("data/summaries_fock3.npy", summaries_fock3)


Loading a model without an environment, this model cannot be trained until it has a valid environment.
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Loading a model without an environment, this model cannot be trained unti